In [ ]:
    # Adapted from https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/03_bonus_embedding-vs-matmul/embeddings-and-linear-layers.ipynb

In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [2]:
# Suppose we have the following 3 training examples,
# which may represent token IDs in a LLM context
idx = tf.constant([2, 3, 1])

# The number of rows in the embedding matrix can be determined
# by obtaining the largest token ID + 1.
# If the highest token ID is 3, then we want 4 rows, for the possible
# token IDs 0, 1, 2, 3
num_idx = tf.reduce_max(idx) + 1

# # The desired embedding dimension is a hyperparameter
out_dim = 5

In [36]:
tf.random.set_seed(123)
fake_weights = np.array([
  [0.00, 0.00, 0.00, 0.00, 0.00], # 0
  [0.11, 0.11, 0.11, 0.11, 0.11], # 1
  [0.22, 0.22, 0.22, 0.22, 0.22], # 2
  [0.33, 0.33, 0.33, 0.33, 0.33], # 3
])
# np_weights = np.array(fake_weights)
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
embedding = tf.keras.layers.Embedding(name='my_embedding', input_dim=num_idx, output_dim=out_dim , weights=[fake_weights])
model = tf.keras.Sequential([embedding])
model.compile()
print("embedding weights:\n", embedding.get_weights(), type(embedding.get_weights()[0]))

embedding weights:
 [array([[0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.11, 0.11, 0.11, 0.11, 0.11],
       [0.22, 0.22, 0.22, 0.22, 0.22],
       [0.33, 0.33, 0.33, 0.33, 0.33]], dtype=float32)] <class 'numpy.ndarray'>


In [37]:
embedding(idx)

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0.22, 0.22, 0.22, 0.22, 0.22],
       [0.33, 0.33, 0.33, 0.33, 0.33],
       [0.11, 0.11, 0.11, 0.11, 0.11]], dtype=float32)>

In [38]:
onehot = tf.one_hot(idx, num_idx)
print("onehot:\n", onehot)

onehot:
 tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]], shape=(3, 4), dtype=float32)


In [53]:
linear = tf.keras.layers.Dense(name='my_dense', units=out_dim, use_bias=False, input_shape=(3,4), weights=[fake_weights])
model = tf.keras.Sequential([linear])
model.compile()
linear.get_weights()

[array([[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.11, 0.11, 0.11, 0.11, 0.11],
        [0.22, 0.22, 0.22, 0.22, 0.22],
        [0.33, 0.33, 0.33, 0.33, 0.33]], dtype=float32)]

In [51]:
linear(onehot)

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0.22, 0.22, 0.22, 0.22, 0.22],
       [0.33, 0.33, 0.33, 0.33, 0.33],
       [0.11, 0.11, 0.11, 0.11, 0.11]], dtype=float32)>